In [89]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import requests
import torch.nn.functional as F
import os


model = models.resnet18(weights="ResNet18_Weights.DEFAULT")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

torch.save(model.state_dict(), 'models/resnet18.pth')
model = models.resnet18()
model.load_state_dict(torch.load('models/resnet18.pth'))
model.to(device)

model.eval()

# load the ImageNet class labels
labels_url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
labels = requests.get(labels_url).json()

def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    preprocessed_image = preprocess(image).unsqueeze(0).to(device)
    return preprocessed_image  # Add batch dimension


def classify_image(image_tensor):
    # Perform inference
    with torch.no_grad():
        outputs = model(image_tensor)

    # Get the predicted class
    _, predicted = torch.max(outputs, 1)

    # Print the predicted class label
    print("Predicted class:", labels[predicted.item()])


image_path = "LPIPS_Images/Lpips_image_9.jpg" 
image = Image.open(image_path)
image_tensor = preprocess_image(image)

classify_image(image_tensor)

d:\programs\miniconda3\envs\rl\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted class: Golden Retriever


In [92]:
# making ground label
def ground_truth(true_class):
    truth_tensor = [0]*1000
    class_index = labels.index(true_class)
    # print("class index of "+ true_class, class_index)
    truth_tensor[class_index] = 1
    truth_tensor = torch.tensor(truth_tensor).unsqueeze(0).to(device)
    truth_tensor = truth_tensor.to(torch.float32)

    return truth_tensor

directory = "LPIPS_Images"
images_path = os.listdir(directory)

for file in images_path:
    file = os.path.join(directory, file)
    image = Image.open(file)
    image_tensor = preprocess_image(image)

    # Perform inference
    with torch.no_grad():
        outputs = model(image_tensor)

    # compute the loss
    ground_truth_tensor = ground_truth('Golden Retriever')
    loss = F.cross_entropy(outputs,ground_truth_tensor)
    print("Prediction loss:", loss.item())



Prediction loss: 0.013039086014032364
Prediction loss: 0.013970794156193733
Prediction loss: 0.017695294693112373
Prediction loss: 0.03191010281443596
Prediction loss: 0.05621420964598656
Prediction loss: 0.06990376859903336
Prediction loss: 0.08865901082754135
Prediction loss: 0.1454923003911972
Prediction loss: 0.39733457565307617
Prediction loss: 0.6536445617675781
